## Roughwork for Problem 2

This section is just to help me understand what the code is doing.

Here I:
- Look at the cube root of individual primes.
- Inspect fractional parts.
- Verify one or two constants manually.


In [8]:
# Exploring how the constant is derived for the prime p = 2.

import numpy as np


p = 2

# Compute the cube root (as float).
cbrt_p = np.cbrt(np.float64(p))
print("cuberoot(2) =", cbrt_p)

# Fractional part of the cube root.
frac_p = cbrt_p - np.floor(cbrt_p)
print("fractional part =", frac_p)

# Multiply fractional part by 2**32.
scaled_p = frac_p * (2**32)
print("frac * 2**32 =", scaled_p)

# Take integer part and show in decimal and hex.
bits_p = int(scaled_p)
print("integer bits =", bits_p)
print("hex constant =", f"0x{bits_p:08x}")


cuberoot(2) = 1.259921049894873
fractional part = 0.25992104989487297
frac * 2**32 = 1116352408.8404636
integer bits = 1116352408
hex constant = 0x428a2f98


In [9]:
def debug_constant_for_prime(p):
    """
    Print all intermediate steps for the constant derived from a prime p.
    This is just for my own understanding and rough checking.
    """
    x = np.float64(p)
    cbrt = np.cbrt(x)
    floor_cbrt = np.floor(cbrt)
    frac = cbrt - floor_cbrt
    scaled = frac * (2**32)
    bits = int(scaled)

    print(f"Prime p              = {p}")
    print(f"cuberoot(p)          = {cbrt}")
    print(f"floor(cuberoot(p))   = {floor_cbrt}")
    print(f"fractional part      = {frac}")
    print(f"fractional * 2**32   = {scaled}")
    print(f"integer bits         = {bits}")
    print(f"hex constant         = 0x{bits:08x}")
    print("-" * 40)


# Try this for a few of the first primes.
for p in [2, 3, 5]:
    debug_constant_for_prime(p)


Prime p              = 2
cuberoot(p)          = 1.259921049894873
floor(cuberoot(p))   = 1.0
fractional part      = 0.25992104989487297
fractional * 2**32   = 1116352408.8404636
integer bits         = 1116352408
hex constant         = 0x428a2f98
----------------------------------------
Prime p              = 3
cuberoot(p)          = 1.4422495703074083
floor(cuberoot(p))   = 1.0
fractional part      = 0.4422495703074083
fractional * 2**32   = 1899447441.1403713
integer bits         = 1899447441
hex constant         = 0x71374491
----------------------------------------
Prime p              = 5
cuberoot(p)          = 1.709975946676697
floor(cuberoot(p))   = 1.0
fractional part      = 0.7099759466766971
fractional * 2**32   = 3049323471.9230537
integer bits         = 3049323471
hex constant         = 0xb5c0fbcf
----------------------------------------


## Roughwork: Problem 3 Padding

In [10]:
msg = b"abc"
print("Original message:", msg)
print("Original bytes:  ", msg)
print("Original hex:    ", msg.hex())
print("Original bit len:", len(msg) * 8)

# Build padded message step by step (similar to block_parse but not as a generator).

bit_len = len(msg) * 8
padded = bytearray(msg)

# Append 0x80 (the '1' bit followed by seven 0 bits).
padded.append(0x80)
print("\nAfter appending 0x80 (hex):")
print(padded.hex())

# Append 0x00 until length ≡ 56 mod 64.
while (len(padded) % 64) != 56:
    padded.append(0x00)

print("\nAfter zero padding (hex):")
print(padded.hex())
print("Length before length-field (bytes):", len(padded))

# Append 64-bit big-endian length.
padded += bit_len.to_bytes(8, "big")

print("\nFinal padded message (hex):")
print(padded.hex())
print("Final padded length (bytes):", len(padded))
print("Final padded length % 64:", len(padded) % 64)

# Show last 8 bytes explicitly.
print("\nLast 8 bytes (hex):", padded[-8:].hex())
print("Decoded bit length:", int.from_bytes(padded[-8:], "big"))


Original message: b'abc'
Original bytes:   b'abc'
Original hex:     616263
Original bit len: 24

After appending 0x80 (hex):
61626380

After zero padding (hex):
6162638000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Length before length-field (bytes): 56

Final padded message (hex):
61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
Final padded length (bytes): 64
Final padded length % 64: 0

Last 8 bytes (hex): 0000000000000018
Decoded bit length: 24
